# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = len(student_data[student_data.passed == 'yes'])
n_failed = len(student_data[student_data.passed == 'no'])
grad_rate = float ( n_passed )/ n_students*100

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows
print y_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
random_state = np.random.randint(0, num_train)
print 'random_state: {}'.format(random_state)

# stratification is enabled during training-test data splitting
# in order to preserve the original class proportions in each dataset
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all,
                                                                     test_size=num_test,
                                                                     train_size=num_train,
                                                                     random_state=random_state,
                                                                     stratify=y_all)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

random_state: 132
Training set: 300 samples
Test set: 95 samples


**Note:** The number of 'yes' to number of 'no' classes ratio is 2:1, therefore unbalanced dataset, as shown below:

In [7]:
print "y_all:\n{}".format(y_all.value_counts())
print "\n"
print "y_train:\n{}".format(y_train.value_counts())
print "\n"
print "y_test:\n{}".format(y_test.value_counts())

y_all:
yes    265
no     130
Name: passed, dtype: int64


y_train:
yes    201
no      99
Name: passed, dtype: int64


y_test:
yes    64
no     31
Name: passed, dtype: int64


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    #print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    #print "Done!"
    print "Training time (secs):      {:.3f}".format(end - start)

In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def predict_labels(clf, features, target):
    #print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    #print "Done!"
    print "Prediction time (secs):    {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes'), confusion_matrix(target, y_pred)

In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size:         {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    (f1_score_train, confusion_matrix_train) = predict_labels(clf, X_train, y_train)    
    print "F1 score for training set: {}".format(f1_score_train)
    print "Confusion matrix for training set:"
    for s in str(confusion_matrix_train).split('\n'):
        print "    {}".format(s)
    f1_score_test, confusion_matrix_test = predict_labels(clf, X_test, y_test)    
    print "F1 score for test set:     {}".format(f1_score_test)
    print "Confusion matrix for test set:"
    for s in str(confusion_matrix_test).split('\n'):
        print "    {}".format(s)

In [11]:
# get training set sizes 100 and 200
X_train_100, y_train_100 = X_train[0:100], y_train[0:100]
X_train_200, y_train_200 = X_train[0:200], y_train[0:200]

### 4.1 Model 1: K-NN Classifier

In [12]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(weights='distance')  # handle unbalanced dataset
print clf
train_predict(clf, X_train_100, y_train_100, X_test, y_test)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
train_predict(clf, X_train, y_train, X_test, y_test)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
------------------------------------------
Training set size:         100
Training time (secs):      0.002
Prediction time (secs):    0.002
F1 score for training set: 1.0
Confusion matrix for training set:
    [[32  0]
     [ 0 68]]
Prediction time (secs):    0.002
F1 score for test set:     0.819444444444
Confusion matrix for test set:
    [[10 21]
     [ 5 59]]
------------------------------------------
Training set size:         200
Training time (secs):      0.001
Prediction time (secs):    0.005
F1 score for training set: 1.0
Confusion matrix for training set:
    [[ 71   0]
     [  0 129]]
Prediction time (secs):    0.003
F1 score for test set:     0.828571428571
Confusion matrix for test set:
    [[13 18]
     [ 6 58]]
------------------------------------------
Training set size:         300
Training time (secs):    

### 4.2 Model 2: Logistic Regression

In [13]:
from sklearn import linear_model
clf2 = linear_model.LogisticRegression(class_weight='balanced',   # handle unbalanced dataset
                                       random_state=random_state)
print clf2
train_predict(clf2, X_train_100, y_train_100, X_test, y_test)
train_predict(clf2, X_train_200, y_train_200, X_test, y_test)
train_predict(clf2, X_train, y_train, X_test, y_test)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=132,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
------------------------------------------
Training set size:         100
Training time (secs):      0.004
Prediction time (secs):    0.001
F1 score for training set: 0.850393700787
Confusion matrix for training set:
    [[27  5]
     [14 54]]
Prediction time (secs):    0.000
F1 score for test set:     0.738461538462
Confusion matrix for test set:
    [[13 18]
     [16 48]]
------------------------------------------
Training set size:         200
Training time (secs):      0.003
Prediction time (secs):    0.000
F1 score for training set: 0.785123966942
Confusion matrix for training set:
    [[53 18]
     [34 95]]
Prediction time (secs):    0.000
F1 score for test set:     0.77519379845
Confusion matrix for test set:
    

### 4.3 Model 3: Support Vector Machine

In [14]:
from sklearn import svm

# from http://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)
# * In SVC, if data for classification are unbalanced (e.g. many positive and few negative),
#   set class_weight='balanced' and/or try different penalty parameters C
# * For SVC, SVR, nuSVC and NuSVR, the size of the kernel cache has a strong impact on run
#   times for larger problems. If you have enough RAM available, it is recommended to set
#   cache_size to a higher value than the default of 200(MB), such as 500(MB) or 1000(MB).
#
clf3 = svm.SVC(class_weight='balanced', cache_size=1000,   
               random_state=random_state)
print clf3
train_predict(clf3, X_train_100, y_train_100, X_test, y_test)
train_predict(clf3, X_train_200, y_train_200, X_test, y_test)
train_predict(clf3, X_train, y_train, X_test, y_test)

SVC(C=1.0, cache_size=1000, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=132, shrinking=True,
  tol=0.001, verbose=False)
------------------------------------------
Training set size:         100
Training time (secs):      0.004
Prediction time (secs):    0.001
F1 score for training set: 0.859504132231
Confusion matrix for training set:
    [[31  1]
     [16 52]]
Prediction time (secs):    0.001
F1 score for test set:     0.7
Confusion matrix for test set:
    [[17 14]
     [22 42]]
------------------------------------------
Training set size:         200
Training time (secs):      0.007
Prediction time (secs):    0.006
F1 score for training set: 0.857142857143
Confusion matrix for training set:
    [[ 56  15]
     [ 21 108]]
Prediction time (secs):    0.008
F1 score for test set:     0.790697674419
Confusion matrix for test set:
    [[17 14]
     [13 51]]
-----------------------

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [18]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search

# use F1 scorer due to unbalanced dataset
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, greater_is_better=True, pos_label='yes')

parameters3 = {'C'            : [x*0.1 for x in range(1,11)],
               'kernel'       : ['rbf', 'linear', 'poly'],
               'degree'       : [x for x in range(0, 11)],
               'gamma'        : ['auto'] + [x*0.1 for x in range(1,11)],
               'class_weight' : ['balanced'],
               'cache_size'   : [1000],
               'random_state' : [random_state]}
clf3_gridsearch = grid_search.GridSearchCV(svm.SVC(), parameters3, scoring=f1_scorer)
train_predict(clf3_gridsearch, X_train, y_train, X_test, y_test)
print clf3_gridsearch.best_estimator_

------------------------------------------
Training set size:         300


/Users/mt_cheng/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Training time (secs):      365.250
Prediction time (secs):    0.010
F1 score for training set: 0.997506234414
Confusion matrix for training set:
    [[ 99   0]
     [  1 200]]
Prediction time (secs):    0.003
F1 score for test set:     0.837837837838
Confusion matrix for test set:
    [[ 9 22]
     [ 2 62]]
SVC(C=1.0, cache_size=1000, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=0, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=132, shrinking=True,
  tol=0.001, verbose=False)
